- *It works!*
- DONE: use `max_batchsize` from `utils`
- DONE: make **dedicated notebook** to (i) compare `conll2003` to `fewnerd` and (ii) to bring `fewnerd` into the same format.
- DONE: use fewnerd
- DONE: get total flops count with **einops**
- DONE: make custom splits
- DONE: reorder the notebook cells:
  - DONE: model (LoRA) and tokenizer (save peftconfig if necessary)
  - DONE: dataset etc
  - DONE: training, metrics and saving (tokenizer and model)
  - DONE: inference via loading saved items (probably tokenizer, model and peftconfig – OR follow this [guide](https://huggingface.co/docs/peft/v0.9.0/en/package_reference/lora#peft.LoraModel))
- DONE: In **Using the fine-tuned model**, [merge and unload](https://huggingface.co/docs/peft/v0.6.2/en/package_reference/tuners#peft.LoraModel.merge_and_unload) or [reinstantiate](https://huggingface.co/docs/peft/v0.6.2/en/task_guides/token-classification-lora#inference) the LoRA model!
- DONE: adjust batch size – and if necessary epochs – for 3000 training steps `num_steps = train_instances*epochs/batch_size` $\geq$ 3000 $\Rightarrow$ `batch_size` $\leq$ `train_instances*epochs/3000 = train_instances/1000` $\Rightarrow$ `batch_size` $\leq$ `train_instances / 1000` for `epochs = 3`<br>DONE: But do it like this:
  - DONE: get max batch size for model (= max_batchsize_by_model)
  - DONE: specify trainig split
  - DONE: get max batch size for training split length (=max_batchsize_by_trainsplit)
  - DONE: impose max batch size of 32
  - DONE: the batch size is the minimum of these three numbers
- DONE: build `results.json` (consider pandas series) via dict. It holds: splits, specified loraconfig details, flops, metrics (per epoch)
- DONE: use the uuid library to save `results.json` under `results_{uuid}.json`
- DONE: declare variable `split` and use it to select splits as well as for logging it in `results_{uuid}.json`.
- DONE: use split `dev` and determine the learning rate for LoRA models. It seems that with `accelerate`, the maximum accepted learning rate is `5e-4` since fails for higher learning rates.
- DONE: outside the training loop make a dummy classification report using scikit-learn with dummy labels and dummy predictions (both formatted as required by classification report)
- DONE: at the end of each epoch, bring the labels and predictions into the required format
- DONE: write a compute metrics function that returns a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report) as dict, then log the relevant values
- DONE: drop the `uuid` from the `logs_dict` filename as it only adds ambiguity about what file to look at.
- check everything once more
- add comments
- make new notebook copy for sweep
- Check once more the wandb [ablation study](https://wandb.ai/ayush-thakur/dl-question-bank/reports/What-s-the-Optimal-Batch-Size-to-Train-a-Neural-Network---VmlldzoyMDkyNDU) on batch sizes!

# NER with `fewnerd` and LoRA

In [1]:
#!pip install scikit-learn==1.4

In [2]:
#!pip list

In [1]:
import os
import re
import json
import time
import torch
import datetime
import itertools
import numpy as np

from peft import LoraConfig, TaskType, PeftModel, PeftConfig, get_peft_model, prepare_model_for_kbit_training
#from uuid import uuid4
from utils import set_seed, make_confusion_matrix
from dotenv import load_dotenv
from datasets import load_dataset, concatenate_datasets, DatasetDict
from evaluate import load
from tqdm.auto import tqdm
from accelerate import Accelerator
from torch.optim import AdamW
from transformers import get_scheduler, pipeline, AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification
from huggingface_hub import login
from sklearn.metrics import classification_report # version 1.3.2 works
from torch.utils.data import DataLoader
from deepspeed.profiling.flops_profiler import FlopsProfiler

load_dotenv()
login(token=os.getenv("HUGGINGFACE_API_KEY"))
logs_dict = {}
logs_dict["seed"] = set_seed(42)

[2024-03-14 21:31:31,300] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/matthias/.cache/huggingface/token
Login successful
All random seeds set to 42.


## Model

In [2]:
list(range(9))

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [3]:
raw_datasets = load_dataset("DFKI-SLT/few-nerd", "supervised")
ner_feature = raw_datasets["train"].features["ner_tags"]
label_names = ner_feature.feature.names
print(label_names)
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}
id2label, label2id

['O', 'art', 'building', 'event', 'location', 'organization', 'other', 'person', 'product']


({'0': 'O',
  '1': 'art',
  '2': 'building',
  '3': 'event',
  '4': 'location',
  '5': 'organization',
  '6': 'other',
  '7': 'person',
  '8': 'product'},
 {'O': '0',
  'art': '1',
  'building': '2',
  'event': '3',
  'location': '4',
  'organization': '5',
  'other': '6',
  'person': '7',
  'product': '8'})

In [4]:
dummy_labels = [
    [0, 0, 1, 8],
    [0, 5, 0, 0, 7]
]
dummy_predictions = [
    [1, 0, 1, 0],
    [0, 5, 0, 4, 4]
]
label_names

['O',
 'art',
 'building',
 'event',
 'location',
 'organization',
 'other',
 'person',
 'product']

In [5]:
make_confusion_matrix(dummy_labels, dummy_predictions, label_names)

{'O': 0, 'art': 1, 'building': 2, 'event': 3, 'location': 4, 'organization': 5, 'other': 6, 'person': 7, 'product': 8}
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]


KeyError: 0

In [ ]:
base_model_id = "FacebookAI/roberta-large"
logs_dict["base_model_id"] = base_model_id
base_model = AutoModelForTokenClassification.from_pretrained(
    base_model_id,
    id2label=id2label,
    label2id=label2id,
    device_map="auto",
    load_in_8bit=True
)
# LoRA model
# datasets:      3 values [1%, 10%, 100%]
# lora_rank:    10 values [1, ..., 512]
# lora_dropout:  5 values [0, 0.1, 0.2, 0.3, 0.4]
# lora_bias:     3 values ["all", "none", "lora_only"]
# => 3 x 10 x 5 x 3 = 450 sweeps per notebook (of which 45 have rank 512 and thus take 45*3.75h=168.75 > 168h=7*24h => a bit over two weeks
# => start with 2 dataset values (10%, 100%) and 3 rank values (2, 8, 32) => 6 sweeps
LoRA_params_dict = {
    "r": 64,
    "target_modules": ["query", "key", "value", "query_proj", "key_proj", "value_proj"],
    "bias": "lora_only",
    "use_rslora": True,
    "task_type": TaskType.TOKEN_CLS,
    "lora_dropout": 0.2
}
logs_dict["LoRA_params_dict"] = LoRA_params_dict
# r =   1 => (  156681, 354476050, 0.00044)
# r =   2 => (  304137, 354623506, 0.00086)
# r =   4 => (  599049, 354918418, 0.00169)
# r =   8 => ( 1188873, 355508242, 0.00334)
# r =  16 => ( 2368521, 356687890, 0.00664)
# r =  32 => ( 4727817, 359047186, 0.01317)
# r =  64 => ( 9446409, 363765778, 0.02597)
# r = 128 => (18883593, 373202962, 0.0506)
# r = 256 => (37757961, 392077330, 0.0963)
# r = 512 => (75506697, 429826066, 0.17567)
config = LoraConfig(
    # GUIDE   => https://huggingface.co/docs/peft/main/en/conceptual_guides/lora#common-lora-parameters-in-peft
    # https://huggingface.co/docs/peft/main/en/conceptual_guides/lora#common-lora-parameters-in-peft:~:text=use_rslora%3A%20When%20set%20to%20True%2C%20uses%20Rank%2DStabilized%20LoRA%20which%20sets%20the%20adapter%20scaling%20factor
    # https://arxiv.org/abs/2312.03732, 
    r = LoRA_params_dict["r"],
    target_modules=LoRA_params_dict["target_modules"],
    bias=LoRA_params_dict["bias"],
    use_rslora=LoRA_params_dict["use_rslora"],
    task_type=LoRA_params_dict["task_type"],
    lora_dropout=LoRA_params_dict["lora_dropout"]
)
logs_dict["LoraConfig"] = str(config)
print(f"LoRA config:\n{config}\n")
adapter_model = prepare_model_for_kbit_training(base_model)
adapter_model = get_peft_model(adapter_model, config)
print(f"base_model type:\n{type(base_model)}\n\nadapter_model type:\n{type(adapter_model)}")
trainable_params, all_params = adapter_model.get_nb_trainable_parameters()
trainable_fraction = round(trainable_params/all_params, 5)
logs_dict["LoRA_model_trainable_params"] = trainable_params
logs_dict["LoRA_model_all_params"] = all_params
logs_dict["LoRA_model_trainable_fraction"] = trainable_fraction
print(f"\ntrainable parameters:\n{trainable_params}")
print(f"\nnall parameters:\n{all_params}")
print(f"\ntrainable fraction:\n{trainable_fraction}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_prefix_space=True)
logs_dict["tokenizer"] = base_model_id
print(f"tokenizer is fast: {tokenizer.is_fast}")
tokenizer

## Dataset

In [ ]:
raw_datasets = load_dataset("DFKI-SLT/few-nerd", "supervised")
raw_datasets

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)
    return new_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

fewnerd_all_processed = concatenate_datasets([raw_datasets["train"], raw_datasets["validation"], raw_datasets["test"]]).map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)
fewnerd_all_processed

In [ ]:
# filter dataset by length if necessary
trainvalid_test_splits = fewnerd_all_processed.train_test_split(test_size=0.15)
test_split_100 = trainvalid_test_splits["test"]
test_split_10 = test_split_100.train_test_split(test_size = 0.1)["test"]
test_split_1 = test_split_100.train_test_split(test_size = 0.01)["test"]
trainvalid_split = trainvalid_test_splits["train"]
train_valid_split = trainvalid_split.train_test_split(test_size=0.15)
valid_split_100 = train_valid_split["test"]
valid_split_10 = valid_split_100.train_test_split(test_size = 0.1)["test"]
valid_split_1 = valid_split_100.train_test_split(test_size = 0.01)["test"]
train_split_100 = train_valid_split["train"]
train_split_10 = train_split_100.train_test_split(test_size = 0.1)["test"]
train_split_1 = train_split_100.train_test_split(test_size = 0.01)["test"]
dev_train_split = train_split_100.train_test_split(test_size = 120)["test"]
dev_valid_split = valid_split_100.train_test_split(test_size = 32)["test"]
dev_test_split = test_split_100.train_test_split(test_size = 8)["test"]
fewnerd_dsd = DatasetDict({
    "train_100": train_split_100,
    "train_10": train_split_10,
    "train_1": train_split_1,
    "valid_100": valid_split_100,
    "valid_10": valid_split_10,
    "valid_1": valid_split_1,
    "test_100": test_split_100,
    "test_10": test_split_10,
    "test_1": test_split_1,
    "train_dev": dev_train_split,
    "valid_dev": dev_valid_split,
    "test_dev": dev_test_split
})
fewnerd_dsd

In [ ]:
split = "dev" # "100", "10", "1", "dev"
assert split in ("100", "10", "1", "dev"), f"Split '{split}' is not a valid choice."
train_split = f"train_{split}"
valid_split = f"valid_{split}"
test_split = f"test_{split}"
train_split, valid_split, test_split

## Training

In [ ]:
batchsize_dict = {
    # values have been found manually via trial and error
    "100": 32,
    "10": 8,
    "1": 1,
    "dev": 1
}
batch_size = batchsize_dict[split]
batch_size

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
data_collator

In [ ]:
batch = data_collator([fewnerd_dsd[train_split][i] for i in [2, 3]])
batch["labels"] # As we can see, the second set of labels has been padded to the length of the first one using -100s.

In [ ]:
train_dataloader = DataLoader(
    fewnerd_dsd[train_split],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size
)
valid_dataloader = DataLoader(fewnerd_dsd[valid_split], collate_fn=data_collator, batch_size=batch_size)
train_dataloader, valid_dataloader

In [ ]:
optimizer = AdamW(adapter_model.parameters(), lr=5e-4) # 5e-4 works
accelerator = Accelerator()
acc_model, optimizer, train_dataloader, valid_dataloader = accelerator.prepare(
    adapter_model,
    optimizer,
    train_dataloader,
    valid_dataloader
)
model_folder = re.sub("/", "-", base_model_id)
output_dir = f"ner_logs/{model_folder}"
output_dir

In [ ]:
num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch
logs_dict["num_training_steps"] = num_training_steps
num_warmup_steps = min(500, round(0.15 * num_update_steps_per_epoch)) # 500 or 15% of one epoch, whichever is less
logs_dict["num_warmup_steps"] = num_warmup_steps
lr_scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)
print(f"training_steps (all epochs):\t{num_training_steps}\nnum_warmup_steps (first epoch):\t{num_warmup_steps}")

In [ ]:
def flatten_list_of_lists(list_of_lists):
    flattened_list = []
    for list_i in list_of_lists:
        list_i = list_i.tolist() # handle tensors in case list_of_lists is a list of tensors
        list_i = list(itertools.chain.from_iterable(list_i)) # flatten list_i using the standard library
        flattened_list += list_i
    return flattened_list

def compute_metrics(y_true, y_pred):
    metrics_dict = classification_report(y_true, y_pred, digits=6, output_dict=True, zero_division="warn")
    accuracy = metrics_dict["accuracy"]
    macroavg_dict = metrics_dict["macro avg"]
    macroavg_dict["accuracy"] = accuracy
    # https://stackoverflow.com/questions/52139110/how-to-change-the-order-of-keys-in-a-python-3-5-dictionary-using-another-list-a
    macroavg_dict_order = {k : macroavg_dict[k] for k in ["accuracy", "precision", "recall", "f1-score", "support"]}
    macroavg_dict_order["support"] = round(macroavg_dict_order["support"]) # 134762.0
    metrics = [f"{key}: {macroavg_dict_order[key]}" for key in macroavg_dict_order.keys()]
    return metrics

prof = FlopsProfiler(acc_model) # deepspeed profiler
flops_list = []
training_start = True
validation_start = True
progress_bar = tqdm(range(num_training_steps))
start_time = time.time() # start time
for epoch in range(num_train_epochs):
    # Training
    acc_model.train()
    prof.start_profile() # start profiling
    for batch in train_dataloader:
        outputs = acc_model(**batch)
        if training_start:
            print("\ntraining")
            print([f"{key} shape: {list(batch[key].shape)}" for key in list(batch.keys())])
            print(f"logits shape: {list(outputs['logits'].shape)}, loss: {float(outputs['loss'])}\n")
            training_start = False
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    prof.stop_profile() # stop profiling
    total_flops = prof.get_total_flops()
    flops_list.append(total_flops)
    # Validation
    acc_model.eval()
    epoch_predictions = []
    epoch_labels = []
    for batch in valid_dataloader:
        with torch.no_grad():
            outputs = acc_model(**batch)
        if validation_start:
            print("validation")
            print([f"{key} shape: {list(batch[key].shape)}" for key in list(batch.keys())])
            print(f"logits shape: {list(outputs['logits'].shape)}, loss: {float(outputs['loss'])}\n")
            validation_start = False
        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        assert len(predictions)==len(labels)
        epoch_predictions.append(predictions)
        epoch_labels.append(labels)
    # collect both, labels and predictions, in one big list each
    flat_epoch_labels = flatten_list_of_lists(epoch_labels)
    flat_epoch_predictions = flatten_list_of_lists(epoch_predictions)
    # get classification report, then log and print results
    metrics = compute_metrics(flat_epoch_labels, flat_epoch_predictions)
    logs_dict[f"epoch_{epoch}_metrics"] = metrics
    print(metrics)
    # # plot confusion matrix
    make_confusion_matrix(epoch_labels, epoch_predictions, label_names)
    # save model
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(acc_model)
    unwrapped_model.save_pretrained(output_dir)
    # save tokenizer
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
#
stop_time = time.time()
training_loop_time = str(datetime.timedelta(seconds = round(stop_time-start_time)))
print(f"training loop time: {training_loop_time}")
logs_dict["training_loop_time"] = training_loop_time
prof.end_profile() # end profiling
logs_dict["flops_list"] = flops_list
print(flops_list)
flops_array = np.array(flops_list)
np.sum(flops_array), np.mean(flops_array)

In [ ]:
set(flat_epoch_labels), set(flat_epoch_predictions), set(flat_epoch_labels)==set(flat_epoch_predictions)

## Save logs

In [ ]:
LoRA_params_dict

In [ ]:
filename = "logsdict"
filename += f"__split={split}"
filename += f"__r={LoRA_params_dict['r']}"
filename += f"__bias={LoRA_params_dict['bias']}"
filename += f"__loradroput=0point{str(LoRA_params_dict['lora_dropout'])[2:]}"
filename

In [ ]:
with open(f"{output_dir}/{filename}.json", "w") as outfile: 
	json.dump(logs_dict, outfile, indent=2)
logs_dict

## Inference

In [ ]:
# load inference model
config = PeftConfig.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)
inference_model = PeftModel.from_pretrained(base_model, output_dir).merge_and_unload()
print(f"type(inference_model):\n{type(inference_model)}\n")
# get inputs from text (source: https://en.wikipedia.org/wiki/Konstanz#History)
text = "Konstanz was the birthplace of Count Ferdinand von Zeppelin, constructor of the famous Zeppelin airships."
inputs = tokenizer(text, return_tensors="pt")
inputs

In [ ]:
# inference
with torch.no_grad():
    logits = inference_model(**inputs).logits
tokens = inputs.tokens()
predictions = torch.argmax(logits, dim=2)
print(f"token (prediction)\n")
for token, prediction in zip(tokens, predictions[0].numpy()):
    print(f"{token} ({id2label[str(prediction)]})")

$\checkmark$